In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install --upgrade spacy==2.1.0
!pip install neuralcoref

     |████████████████████████████████| 27.7MB 111kB/s 
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9
     |████████████████████████████████| 296kB 3.4MB/s 


In [0]:
import neuralcoref
import pandas as pd
import spacy


100%|██████████| 40155833/40155833 [00:00<00:00, 53040437.14B/s]


In [0]:
def lowercase_if_not_entity(span):
    first_token = span[0]
    if first_token.ent_type_ == '' and first_token.is_sent_start and first_token.text.istitle():
        return span.text[0].lower() + span.text[1:]
    return span.text

def check_possessive(token, sub):
    if sub in possessives:
        return ''
    text = token.text.lower()
    if text in possessives:
        if sub[-1] == 's':
            return "'"
        else:
            return "'s"
    return ''

def get_correct_case(coref, main_uncased, main_original):
    if coref[0].is_sent_start:
        return main_original.text[0].upper() + main_original.text[1:]
    else:
        return main_uncased

In [0]:
possessives = {'his', 'her', 'its', 'their', 'hers', 'theirs'}

def get_resolved(doc, clusters):
    ''' modified based on https://github.com/huggingface/neuralcoref/blob/master/neuralcoref/neuralcoref.pyx'''
    ''' Return a list of utterrances text where the coref are resolved to the most representative mention'''
    resolved = list(tok.text_with_ws for tok in doc)
    for cluster in clusters:
        for coref in cluster:
            main_mention = lowercase_if_not_entity(cluster.main)
            if coref != cluster.main:
                resolved[coref.start] = get_correct_case(coref, main_mention, cluster.main) + check_possessive(
                    coref, main_mention) + doc[coref.end - 1].whitespace_
                for i in range(coref.start + 1, coref.end):
                    resolved[i] = ""
    return ''.join(resolved)

In [0]:
nlp = spacy.load('en_core_web_sm')

In [0]:
# Add neural coref to SpaCy's pipe
neuralcoref.add_to_pipe(nlp, blacklist=False, max_dist=10, max_dist_match=10)

In [0]:
root_dir = "/content/drive/My Drive/Polygence/Audrey/"

In [0]:
df = pd.read_csv(root_dir + "data/all_articles.csv")
df.head()

,Unnamed: 0,source,author,title,url,publishedAt,content
0,0,abc-news,John Verhovek,5 takeaways from the 2nd night of the Democrat...,https://abcnews.go.com/Politics/takeaways-2nd-...,2019-08-01T03:31:25Z,\nEven before all the candidates took the stag...
1,1,abc-news,ABC NEWS,Fact-checking Democratic candidates on the iss...,https://abcnews.go.com/Politics/democratic-deb...,2019-08-01T01:29:38Z,\nHere's ABC News' fact check of the second of...
2,2,abc-news,Kendall Karson and Elizabeth Thomas,What to expect from 2nd night of Democratic de...,https://abcnews.go.com/Politics/democratic-deb...,2019-07-31T20:01:36Z,\nAfter Sens. Bernie Sanders and Elizabeth War...
3,3,abc-news,Soo Rin Kim,2020 candidates spend millions on online ads a...,https://abcnews.go.com/Politics/love-hate-rela...,2019-07-27T10:22:00Z,\nDemocrats vying for presidential election in...
4,4,abc-news,Rick Klein and MaryAlice Parks,The Note: Blue wave continues ride through Ken...,https://abcnews.go.com/Politics/note-blue-wave...,2019-11-06T11:05:10Z,\nThe TAKE with Rick Klein\n \nIt might just b...


In [0]:
df.iloc[0]["content"]

'\nEven before all the candidates took the stage in Detroit on Wednesday for the second night of the latest Democratic primary debate, the looming conflict at center stage was already apparent.\n \n"Go easy on me kid," former Vice President Joe Biden said with a smile to California Sen. Kamala Harris, who he sparred with in last month\'s debate. Night two also put the diversity of the Democratic Party on display, with all of the candidates of color running for the Democratic presidential nomination taking the stage to pitch themselves as the candidate that can defeat President Donald Trump. \nBoth Harris and Biden were targets of constant attacks from rivals across the stage Wednesday night, with New Jersey Sen. Cory Booker, New York City Mayor Bill de Blasio, Hawaii Congresswoman Tulsi Gabbard and Washington Gov. Jay Inslee all taking aim at the two polling frontrunners over their records and their plan to lead the Democrats to victory against President Trump in 2020. \nBooker sharply

In [0]:
def process_article(article):
    doc = nlp(article)
    print(len(article))
    #return get_resolved(doc, doc._.coref_clusters)
    return doc._.coref_resolved

In [0]:
batch_size=100
i = 33 * batch_size
count = 34
while i < (len(df) - batch_size):
    print(count)
    chunk = df[i:i+batch_size]
    chunk["coref_resolved"] = chunk["content"].astype(str).apply(process_article)
    chunk.to_csv(root_dir + "data/coref_resolved/coref_resolved_" + str(count) + ".csv", index=False)
    count+=1
    i+=batch_size

34
2373
12895
13217
2748
11245
13425
52078
13425
74
13191
13597
74
74
11681
74
74
74
13449
12414
14476
11635
18870
13993
12424
13116
14771
21569
13553
8884
74
74
74
74
74
74
74
74
74
74
12942
20264
12723
74
74
74
74
74
74
74
12611
74
12139
16579
74
74
74
74
74
74
7875
2247
54209
3109
3109
11651
1216
14918
9927
74
74
74
53214
8329
2872
8517
13042
12340
11795
74
12574
11421
5254
12232
12717
74
74
2782
11027
13828
13828
14940
12813
2022
11344
3414
12416
11485
4721
49679
74


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


35
10176
11342
74
74
74
74
11962
12694
15893
74
12470
12525
13902
74
74
74
74
12918
18409
74
74
74
74
74
12560
74
14269
3348
3932
10818
74
74
74
13988
3862
74
3231
3207
74
6335
10669
10820
74
2543
16425
6218
13581
74
74
74
74
74
74
74
74
10047
12846
2121
2580
14170
12404
12026
9643
15230
6841
12860
74
11230
11960
74
74
74
74
74
12055
74
74
12892
12818
12745
16000
74
74
74
74
11906
74
17274
11011
74
74
74
17172
74
18747
13902
12066
11778
18442
13177
36
12649
14881
74
74
74
74
74
74
74
74
74
74
74
16278
13783
11283
74
74
74
74
74
74
14042
15408
18321
20891
13783
74
74
74
12461
1377
9068
74
74
11603
10144
12841
12298
13365
16125
15722
11603
74
4252
74
74
74
74
74
74
74
74
74
12125
11373
5938
8815
74
74
74
12152
17983
17983
1747
12187
74
74
74
74
74
74
74
74
74
74
74
74
12714
12714
13900
1860
11955
74
74
74
74
74
74
12358
12773
74
14886
5705
5322
15123
15453
74
74
74
37
74
74
13593
74
74
15945
74
74
12264
74
13882
13002
11605
3528
3343
4465
14541
3483
4103
3045
1960
6766
2831
3480
3650
517

In [0]:
# process last chunk
chunk = df[(count * batch_size):]
chunk["coref_resolved"] = chunk["content"].astype(str).apply(process_article)
chunk.to_csv(root_dir + "data/coref_resolved/coref_resolved_" + str(count) + ".csv", index=False)

4829
5959


KeyboardInterrupt: ignored

In [0]:
import glob

In [0]:
dfs = []
for f in glob.glob(root_dir + "data/coref_resolved/coref_resolved_*.csv"):
    dfs.append(pd.read_csv(f))
concat = pd.concat(dfs)
concat.drop_duplicates("url", inplace=True)

In [0]:
len(concat)

In [0]:
len(df)

In [0]:
concat.to_csv(root_dir + "data/coref_resolved_all.csv", index=False)